# Demonstration of Full ML Pipeline with a Materials Science Database

We will use Matminer to fetch materials datasets that are ML-ready (i.e., they largely don't require any data cleaning step). We will then consider two featurization approaches: compositional (using Matminer again) and structural (using DScribe). Then we will remove low variance features and highly correlated features with Pandas. Lastly, we use Scikit-Learn for data splitting, recursive feature elimintation, model performance metrics calculations, and ML model training (random forest model).

In [ ]:
from matminer.datasets.dataset_retrieval import load_dataset, get_all_dataset_info
from matminer.datasets import get_available_datasets

print(*get_available_datasets(print_format='short'), sep='\n')

For this demonstration let us work with the Matbench perovskite dataset which contains 18,928 DFT-calculated formation energies (in eV/unit cell).

In [ ]:
print(get_all_dataset_info('matbench_perovskites'))

## 1. Dataset Analysis

In [ ]:
import pandas as pd
df_full = load_dataset('matbench_perovskites')
df_full.describe()  # type: ignore

For computational efficiency sake, we will randomly sample a smaller number of data points.

In [ ]:
n_sample = 2000
df = df_full.sample(n=2000, random_state=17)  # subsample for speed   # type: ignore
df.describe()

Now, let us have a look at the distribution of the larget property `e_form`.

In [ ]:
import matplotlib.pyplot as plt

df['e_form'].hist(bins=30)
plt.xlabel('Formation Energy (eV/unit cell)')
plt.ylabel('Frequency')
plt.title('Histogram of Formation Energies')
plt.show()

## 2. Composition-based Features
First, let us add a new column that contains the composition of each entry through converting the Pymatgen structure to a composition.

In [ ]:
df_comp = df.copy()
df_comp['composition'] = df_comp.structure.apply(lambda x: x.composition )
df_comp.head()

### 2.1 Featurization
Then we will featurize the compositions using the Matminer `ElementProperty` featurizer and use the magpie preset.

In [ ]:
from matminer.featurizers.composition.composite import ElementProperty

el_prop_featuriser = ElementProperty.from_preset(preset_name='magpie', impute_nan=False)
el_prop_featuriser.set_n_jobs(1)
df_featurized = el_prop_featuriser.featurize_dataframe(df_comp, col_id='composition')

print(df_featurized.shape)  # type: ignore
print(df_featurized.isnull().sum().sum())  # Check for any NaN values  # type: ignore
df_featurized.head()  # type: ignore

### 2.2 Feature Engineering and Cleaning
We store the target column is `"e_form"` in variable `y`, and the features in `X_all`.

We remove all features with very small variance and features that highly correlate with other features in the dataset.

In [ ]:
y = df_featurized['e_form']  # type: ignore
X_all = df_featurized.drop(columns=['e_form', 'structure', 'composition'])  # type: ignore

print("Number of features before cleaning:", X_all.shape[1])

# Identify columns with very small variance and drop them
small_var_cols = X_all.columns[X_all.var() < 1e-5].tolist()
print("Columns with very small variance:", small_var_cols)
X_all = X_all.drop(columns=small_var_cols)
corr_matrix = X_all.corr(method='pearson')
print("Number of features after removing small variance columns:", X_all.shape[1])

# Remove highly correlated columns
threshold = 0.99
to_drop = set()
for col in corr_matrix.columns:
    high_corr = corr_matrix.index[(corr_matrix[col].abs() > threshold) & (corr_matrix.index != col)]
    to_drop.update(high_corr)
print("Columns to drop due to high correlation:", to_drop)
X = X_all.drop(columns=list(to_drop))
print("Number of features after removing highly correlated columns:", X.shape[1])

# For the remaining features, let's visualize the correlation matrix
corr_matrix = X.corr(method='pearson')

plt.figure(figsize=(16, 12))
im = plt.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('Pearson Correlation Matrix')
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90, fontsize=6)
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns, fontsize=6)
plt.tight_layout()
plt.show()

### 2.3 Feature Scaling and Dataset Splitting
Now we use the `StandardScaler` of the Scikit-Learn package, which is always advisable for all input features as it improves numeric stability during model optimization (e.g., gradient descent).

Then we split the dataset into training, validation, and test sets.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

test_fraction = 0.1
validation_fraction = 0.2
X_trainval, X_test, y_trainval, y_test = train_test_split(X_scaled, y, 
                                                          test_size=test_fraction, 
                                                          random_state=17)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, 
                                                  test_size=validation_fraction/(1-test_fraction), 
                                                  random_state=17)
print(f"Training fraction: {X_train.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Validation fraction: {X_val.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Test fraction: {X_test.shape[0] / X_scaled.shape[0]:.2f}")



### 2.4 Dummy Baseline Model

We will compute the MAE for a dummy baseline model, which always predicts the average of the training set target value for all validation predictions. This is used as a reference to compare how well the model learns compared to a very naive "model".

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_train = y_train.mean()
baseline_mae = mean_absolute_error(y_val, [mean_train] * len(y_val))

print(f"Baseline MAE (predicting mean formation energy): {baseline_mae:.4f} eV")

### 2.5 Hyperparameter Optimization

Here, we will choose the `RandomForestRegressor` of Scikit-Learn which has a few hyperparameters. We will optimize only one here for demonstration purposes: `n_estimators`.



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from tqdm.notebook import tqdm

n_estimators_list = [10, 50, 100, 500]
train_maes = []
val_maes = []

for n in tqdm(n_estimators_list, desc='Training RF models'):
    rf = RandomForestRegressor(n_estimators=n, random_state=17, n_jobs=1)
    rf.fit(X_train, y_train)
    y_train_pred = rf.predict(X_train)
    y_val_pred = rf.predict(X_val)
    train_maes.append(mean_absolute_error(y_train, y_train_pred))
    val_maes.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8, 5))
plt.plot(n_estimators_list, train_maes, marker='o', label='Train MAE')
plt.plot(n_estimators_list, val_maes, marker='o', label='Validation MAE')
plt.axhline(baseline_mae, color='gray', linestyle='--', label='Mean Baseline')
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Absolute Error')
plt.title('Random Forest: MAE vs. Number of Estimators')
plt.legend()
plt.tight_layout()
plt.show()

### 2.6 Recursive Feature Elimintation

Now we use the RF model to further downselect important features by using the `RFE` class of Scikit-Learn.

*Remark*: This step can get quite compute-intensive! Here, we check RFE in steps of 5 features. Ideally, this should be carried out in steps of 1.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm

rf = RandomForestRegressor(n_estimators=50, random_state=17, n_jobs=1)
n_features_list = list(range(5, X_val.shape[1]+1, 5))
val_errors = []
train_errors = []
selected_features_dict = {}

for n_features in tqdm(n_features_list, desc='RFE Progress'):
    rfe = RFE(estimator=rf, n_features_to_select=n_features, step=5)
    rfe.fit(X_train, y_train)
    selected_features_dict[n_features] = list(X.columns[rfe.support_])
    X_train_rfe = rfe.transform(X_train)
    rf.fit(X_train_rfe, y_train)
    y_train_pred = rf.predict(X_train_rfe)
    train_errors.append(mean_absolute_error(y_train, y_train_pred))
    X_val_rfe = rfe.transform(X_val)
    y_val_pred = rf.predict(X_val_rfe)
    val_errors.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8, 5))
plt.plot(n_features_list, train_errors, label='Train MAE')
plt.plot(n_features_list, val_errors, label='Validation MAE')
plt.xlabel('Number of Selected Features')
plt.ylabel('Mean Absolute Error')
plt.title('RFE: MAE vs. Number of Features')
plt.legend()
plt.tight_layout()
plt.show()

### 2.7 Final Model Evaluation
Lastly, we compute all train/validation/test metrics (here we look at MAE and R^2, but this can be easily extended to RMSE, MAPE, etc.). We also create parity plots (true vs. predicted target values) for all three sets.

In [ ]:
from sklearn.metrics import r2_score

final_features = selected_features_dict[30]
rf_final = RandomForestRegressor(n_estimators=100, random_state=17, n_jobs=1)
X_train_final = X_train[:, [X.columns.get_loc(f) for f in final_features]]
rf_final.fit(X_train_final, y_train)

# Predict on train, validation, and test sets
X_val_final = X_val[:, [X.columns.get_loc(f) for f in final_features]]
X_test_final = X_test[:, [X.columns.get_loc(f) for f in final_features]]

y_train_pred = rf_final.predict(X_train_final)
y_val_pred = rf_final.predict(X_val_final)
y_test_pred = rf_final.predict(X_test_final)

# Calculate metrics
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=True)
min_val = 0.0
max_val = 5.0

# Train parity plot
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[0].set_title(f'Train (MAE={mae_train:.3f}, R2={r2_train:.3f})')
axes[0].set_xlabel('True Formation Energy')
axes[0].set_ylabel('Predicted Formation Energy')
axes[0].set_aspect('equal', adjustable='box')

# Validation parity plot
axes[1].scatter(y_val, y_val_pred, alpha=0.5, color='orange')
axes[1].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[1].set_title(f'Validation (MAE={mae_val:.3f}, R2={r2_val:.3f})')
axes[1].set_xlabel('True Formation Energy')
axes[1].set_ylabel('Predicted Formation Energy')
axes[1].set_aspect('equal', adjustable='box')

# Test parity plot
axes[2].scatter(y_test, y_test_pred, alpha=0.5, color='green')
axes[2].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[2].set_title(f'Test (MAE={mae_test:.3f}, R2={r2_test:.3f})')
axes[2].set_xlabel('True Formation Energy')
axes[2].set_ylabel('Predicted Formation Energy')
axes[2].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

## 3. Structure-based Features

Now, let us use the same steps but instead of using compositional featurization, we will use a strucutral one (the Ewald sum matrix approach via the DScribe package).

### 3.1 Featurization

In [ ]:
from dscribe.descriptors import EwaldSumMatrix
from pymatgen.io.ase import AseAtomsAdaptor
import numpy as np

# Determine the maximum number of atoms across all structures in the dataset
n_max = 0
for mat in df['structure']:
    if len(mat) > n_max :
        n_max = len(mat)
print(n_max)

ews = EwaldSumMatrix(n_atoms_max=n_max, permutation="eigenspectrum")

ase_structures = [AseAtomsAdaptor.get_atoms(struc) for struc in df['structure']]
ews_matrices = np.array(ews.create(ase_structures))

In [ ]:
ews_columns = [f'ews_{i}' for i in range(ews_matrices.shape[1])]
df_featurized_ews = df.copy()
df_featurized_ews[ews_columns] = pd.DataFrame(ews_matrices, index=df_featurized_ews.index)
print(df_featurized_ews.head())

### 3.2 Feature Engineering and Cleaning

In [ ]:
y = df_featurized_ews['e_form']
X_all_struc = df_featurized_ews.drop(columns=['e_form', 'structure'])

print("Number of features before cleaning:", X_all_struc.shape[1])

# Identify columns with very small variance and drop them
small_var_cols_struc = X_all_struc.columns[X_all_struc.var() < 1e-5].tolist()
print("Columns with very small variance:", small_var_cols_struc)
X_all_struc = X_all_struc.drop(columns=small_var_cols_struc)
corr_matrix_struc = X_all_struc.corr(method='pearson')
print("Number of features after removing small variance columns:", X_all_struc.shape[1])

# Remove highly correlated columns
threshold = 0.99
to_drop_struc = set()
for col in corr_matrix_struc.columns:
    high_corr_struc = corr_matrix_struc.index[(corr_matrix_struc[col].abs() > threshold) & (corr_matrix_struc.index != col)]
    to_drop_struc.update(high_corr_struc)
print("Columns to drop due to high correlation:", to_drop_struc)
X_struc = X_all_struc.drop(columns=list(to_drop_struc))
print("Number of features after removing highly correlated columns:", X_struc.shape[1])

corr_matrix_struc = X_struc.corr(method='pearson')

plt.figure(figsize=(16, 12))
im = plt.imshow(corr_matrix_struc, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.title('Pearson Correlation Matrix')
plt.xticks(range(len(corr_matrix_struc.columns)), [str(c) for c in corr_matrix_struc.columns], rotation=90, fontsize=6)
plt.yticks(range(len(corr_matrix_struc.columns)), [str(c) for c in corr_matrix_struc.columns], fontsize=6)
plt.tight_layout()
plt.show()

### 3.3 Feature Scaling and Dataset Splitting

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_struc)

test_fraction = 0.1
validation_fraction = 0.2
X_trainval, X_test, y_trainval, y_test = train_test_split(X_scaled, y, 
                                                          test_size=test_fraction, 
                                                          random_state=17)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, 
                                                  test_size=validation_fraction/(1-test_fraction), 
                                                  random_state=17)
print(f"Training fraction: {X_train.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Validation fraction: {X_val.shape[0] / X_scaled.shape[0]:.2f}")
print(f"Test fraction: {X_test.shape[0] / X_scaled.shape[0]:.2f}")

### 3.4 Dummy Baseline Model

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_train = y_train.mean()
baseline_mae = mean_absolute_error(y_val, [mean_train] * len(y_val))
print(f"Baseline MAE (predicting mean formation energy): {baseline_mae:.4f} eV")

### 3.5 Hyperparameter Optimization

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from tqdm.notebook import tqdm

n_estimators_list = [10, 50, 100, 500]
train_maes = []
val_maes = []

for n in tqdm(n_estimators_list, desc='Training RF models'):
    rf = RandomForestRegressor(n_estimators=n, random_state=17, n_jobs=1)
    rf.fit(X_train, y_train)
    y_train_pred = rf.predict(X_train)
    y_val_pred = rf.predict(X_val)
    train_maes.append(mean_absolute_error(y_train, y_train_pred))
    val_maes.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8, 5))
plt.plot(n_estimators_list, train_maes, marker='o', label='Train MAE')
plt.plot(n_estimators_list, val_maes, marker='o', label='Validation MAE')
plt.axhline(baseline_mae, color='gray', linestyle='--', label='Mean Baseline')
plt.xlabel('Number of Estimators')
plt.ylabel('Mean Absolute Error')
plt.title('Random Forest: MAE vs. Number of Estimators')
plt.legend()
plt.tight_layout()
plt.show()

### 2.6 Recursive Feature Elimintation
Technically, this step is not required because we already start off with a low number of features to begin with.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_absolute_error
from tqdm.notebook import tqdm

rf = RandomForestRegressor(n_estimators=50, random_state=17, n_jobs=1)
n_features_list = list(range(1, X_val.shape[1]+1, 1))
val_errors = []
train_errors = []
selected_features_dict = {}

for n_features in tqdm(n_features_list, desc='RFE Progress'):
    rfe = RFE(estimator=rf, n_features_to_select=n_features, step=1)
    rfe.fit(X_train, y_train)
    selected_features_dict[n_features] = list(X_struc.columns[rfe.support_])
    X_train_rfe = rfe.transform(X_train)
    rf.fit(X_train_rfe, y_train)
    y_train_pred = rf.predict(X_train_rfe)
    train_errors.append(mean_absolute_error(y_train, y_train_pred))
    X_val_rfe = rfe.transform(X_val)
    y_val_pred = rf.predict(X_val_rfe)
    val_errors.append(mean_absolute_error(y_val, y_val_pred))

plt.figure(figsize=(8, 5))
plt.plot(n_features_list, train_errors, label='Train MAE')
plt.plot(n_features_list, val_errors, label='Validation MAE')
plt.xlabel('Number of Selected Features')
plt.ylabel('Mean Absolute Error')
plt.title('RFE: MAE vs. Number of Features')
plt.legend()
plt.tight_layout()
plt.show()

### 3.7 Final Model Evaluation

In [ ]:
from sklearn.metrics import r2_score

rf_final = RandomForestRegressor(n_estimators=100, random_state=17, n_jobs=1)
rf_final.fit(X_train, y_train)

y_train_pred = rf_final.predict(X_train)
y_val_pred = rf_final.predict(X_val)
y_test_pred = rf_final.predict(X_test)

# Calculate metrics
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)
r2_test = r2_score(y_test, y_test_pred)

fig, axes = plt.subplots(1, 3, figsize=(18, 6), sharex=True, sharey=True)
min_val = 0.0
max_val = 5.0

# Train parity plot
axes[0].scatter(y_train, y_train_pred, alpha=0.5, color='blue')
axes[0].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[0].set_title(f'Train (MAE={mae_train:.3f}, R2={r2_train:.3f})')
axes[0].set_xlabel('True Formation Energy')
axes[0].set_ylabel('Predicted Formation Energy')
axes[0].set_aspect('equal', adjustable='box')

# Validation parity plot
axes[1].scatter(y_val, y_val_pred, alpha=0.5, color='orange')
axes[1].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[1].set_title(f'Validation (MAE={mae_val:.3f}, R2={r2_val:.3f})')
axes[1].set_xlabel('True Formation Energy')
axes[1].set_ylabel('Predicted Formation Energy')
axes[1].set_aspect('equal', adjustable='box')

# Test parity plot
axes[2].scatter(y_test, y_test_pred, alpha=0.5, color='green')
axes[2].plot([min_val, max_val], [min_val, max_val], 'k--', lw=2)
axes[2].set_title(f'Test (MAE={mae_test:.3f}, R2={r2_test:.3f})')
axes[2].set_xlabel('True Formation Energy')
axes[2].set_ylabel('Predicted Formation Energy')
axes[2].set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.show()

**Conclusion**: 5 structural features outperformed 100+ compositional features! 

There is still for improvement though. Top compositional features could be added to the 5 structural features. Alternatively, more sophisticated approaches (graph-based) may perform better here.

## 4. Exercise 11.1 → Homework 4

Now, it's your turn to utilize this entire pipeline on a different Matminer dataset! However, notice that some datasets only have compositions available and not the structures. Do it twice, once for a compositional featurization and once for a structural one (or, if your dataset does not contain structures, use a second compositional featurization approach).

Please, fill in the details in this [Google Sheet](https://docs.google.com/spreadsheets/d/1xbT4lRMYQGrhBQGFczFD5wscrriwWQGf82Gv9AnDkbA/4).

In [ ]:
# List of available datasets:
from matminer.datasets import get_available_datasets

print(*get_available_datasets(print_format='short'), sep='\n')

List of available Matminer datasets: [link](https://hackingmaterials.lbl.gov/matminer/dataset_summary.html)

List of available Matminer featurizers: [link](https://hackingmaterials.lbl.gov/matminer/featurizer_summary.html)

List of available DScribe featurizers: [link](https://singroup.github.io/dscribe/latest/)